### Tools
Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:
1. A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
2. A function or coroutine to execute.

In [1]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response

AIMessage(content='<think>\nOkay, so the user is asking why parrots talk. Let me start by recalling what I know about parrots. Parrots are known for their ability to mimic human speech, right? But why do they do that? I need to break this down into possible reasons.\n\nFirst, I remember that in the wild, parrots might use vocalizations to communicate with each other. Maybe talking is a learned behavior from humans that serves a similar purpose. For example, a pet parrot might talk to interact with its owner, get attention, or even get food. That makes sense because animals often repeat behaviors that lead to positive outcomes.\n\nThen there\'s the aspect of intelligence. Parrots are considered highly intelligent birds. They can learn complex words and phrases, which suggests that their ability to talk isn\'t just mimicry but might involve some understanding. But how does that tie into why they do it? Maybe they talk because they enjoy the interaction and the mental stimulation.\n\nAnot

#### Create a tool: is similar to create an agent that we learned in part 2

In [4]:
from langchain.tools import tool

@tool
def get_weather(location: str)-> str:
    # below line is required as LLM use it to define the functionality of tool
    """Get the weather at a location""" 
    return f"It's sunny in {location}"

model_with_tools = model.bind_tools([get_weather])

In [5]:
response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking about the weather in Boston. I need to use the get_weather function. The function requires a location parameter. Boston is the location here. So I should call get_weather with location set to "Boston". Let me make sure there\'s no typo. Everything looks good. Let me format the tool call correctly.\n', 'tool_calls': [{'id': 'nrgp0b19e', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 154, 'total_tokens': 246, 'completion_time': 0.145296406, 'completion_tokens_details': {'reasoning_tokens': 68}, 'prompt_time': 0.007528255, 'prompt_tokens_details': None, 'queue_time': 0.170372135, 'total_time': 0.152824661}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_2bfcc54d36', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_ru

### Tool Execution Loops

In [6]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

The weather in Boston is sunny.


In [7]:
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Boston. I need to use the get_weather function. The function requires a location parameter. Boston is the location here. So I should call get_weather with location set to "Boston". Let me make sure there\'s no typo. Everything looks good. I\'ll format the tool call as specified.\n', 'tool_calls': [{'id': '5a86hs9ck', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 153, 'total_tokens': 246, 'completion_time': 0.148331614, 'completion_tokens_details': {'reasoning_tokens': 69}, 'prompt_time': 0.006322835, 'prompt_tokens_details': None, 'queue_time': 0.069543422, 'total_time': 0.154654449}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_d58dbe76cd', 'service_tier': 'on_demand', 'finish_